# Patent Analysis

In [51]:
!pip install -r requirements.txt


[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Libraries

In [45]:
import sys
import importlib
sys.path.append('src')

import preprocess_data
importlib.reload(preprocess_data)

<module 'preprocess_data' from 'c:\\Users\\armin\\Documents\\GitRepo\\uspto_patent_analysis\\src\\preprocess_data.py'>

## Downloading patents and extracting individual XML file

In [46]:
year = 2023
month = 1
day = 5

In [47]:
download_status = preprocess_data.download_weekly_patents(year, month, day)

Building the URL...
URL constructed: https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2023/ipa230105.zip
Requesting the file...
File retrieved successfully. Starting download...
File downloaded successfully. Starting extraction...
File extracted successfully.
ZIP file c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\patents.zip deleted after extraction.


In [48]:
extract_status = preprocess_data.extract_patents(year, month, day)

Locating the patent file...
Reading the patent file...
Splitting the patent file into individual patents...
Total patents found: 7824
Creating directory to store individual patents...
Writing individual patents to separate XML files...
Patent extraction complete.
Main XML file c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105.xml deleted after extraction.


In [49]:
if download_status:
    print("### Patent download was completed.")
if extract_status:
    print("### Patent extraction was completed.")

### Patent download was completed.


## Updating requirments file

In [50]:
!pip freeze > requirements.txt